In [ ]:
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import load_model

class HandGestureRecognition:
    def __init__(self, model_path, actions, seq_length=30):
        self.actions = actions
        self.seq_length = seq_length
        self.model = load_model(model_path)

        # Initialize MediaPipe Hands
        self.mp_hands = mp.solutions.hands
        self.mp_drawing = mp.solutions.drawing_utils
        self.hands = self.mp_hands.Hands(
            max_num_hands=1,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5
        )

        # Initialize variables
        self.seq = []
        self.action_seq = []
        
    def preprocess_frame(self, frame):
        flipped_frame = cv2.flip(frame, 1)
        rgb_frame = cv2.cvtColor(flipped_frame, cv2.COLOR_BGR2RGB)
        return rgb_frame

    def process_frame(self, frame):
        result = self.hands.process(frame)
        return result

    def extract_joint_angles(self, hand_landmarks):
        joint = np.zeros((21, 4))
        for j, lm in enumerate(hand_landmarks):
            joint[j] = [lm.x, lm.y, lm.z, lm.visibility]

        v1 = joint[[0, 1, 2, 3, 0, 5, 6, 7, 0, 9, 10, 11, 0, 13, 14, 15, 0, 17, 18, 19], :3]  # Parent joints
        v2 = joint[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], :3]  # Child joints
        v = v2 - v1
        v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

        angle = np.arccos(np.einsum('nt,nt->n',
                                    v[[0, 1, 2, 4, 5, 6, 8, 9, 10, 12, 13, 14, 16, 17, 18], :],
                                    v[[1, 2, 3, 5, 6, 7, 9, 10, 11, 13, 14, 15, 17, 18, 19], :]))
        angle = np.degrees(angle)
        return np.concatenate([joint.flatten(), angle])

    def predict_action(self, input_data):
        input_data = np.expand_dims(np.array(input_data, dtype=np.float32), axis=0)
        y_pred = self.model.predict(input_data).squeeze()
        i_pred = int(np.argmax(y_pred))
        conf = y_pred[i_pred]
        return i_pred, conf

    def draw_landmarks(self, frame, hand_landmarks):
        self.mp_drawing.draw_landmarks(frame, hand_landmarks, self.mp_hands.HAND_CONNECTIONS)

    def run(self):
        cap = cv2.VideoCapture(0)
        
        while cap.isOpened():
            ret, img = cap.read()
            if not ret:
                break

            img_rgb = self.preprocess_frame(img)
            result = self.process_frame(img_rgb)

            if result.multi_hand_landmarks:
                for res in result.multi_hand_landmarks:
                    d = self.extract_joint_angles(res.landmark)
                    self.seq.append(d)

                    if len(self.seq) < self.seq_length:
                        continue

                    input_data = self.seq[-self.seq_length:]
                    i_pred, conf = self.predict_action(input_data)

                    if conf < 0.9:
                        continue

                    action = self.actions[i_pred]
                    self.action_seq.append(action)

                    if len(self.action_seq) < 3:
                        continue

                    this_action = '?' if self.action_seq[-1] != self.action_seq[-2] or self.action_seq[-1] != self.action_seq[-3] else action

                    cv2.putText(img, f'{this_action.upper()}', 
                                org=(int(res.landmark[0].x * img.shape[1]), 
                                     int(res.landmark[0].y * img.shape[0] + 20)), 
                                fontFace=cv2.FONT_HERSHEY_SIMPLEX, 
                                fontScale=1, 
                                color=(255, 255, 255), 
                                thickness=2)

                    self.draw_landmarks(img, res)

            cv2.imshow('Hand Gesture Recognition', img)

            if cv2.waitKey(1) == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()

# Example usage
if __name__ == "__main__":
    actions = ['good', 'five', 'fit', 'V', 'piece']
    model_path = r'C:\Users\user\Desktop\python\ModuleProject-main\model.keras'
    recognizer = HandGestureRecognition(model_path, actions)
    recognizer.run()

In [1]:
import numpy as np
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [6]:
good_data = np.load('C:/Users/user/Desktop/python/ModuleProject-main/dataset/seq_good_1736319783.npy')
five_data = np.load('C:/Users/user/Desktop/python/ModuleProject-main/dataset/seq_five_1736319783.npy')
fit_data = np.load('C:/Users/user/Desktop/python/ModuleProject-main/dataset/seq_seq_fit_1736319783.npy')
V_data = np.load('C:/Users/user/Desktop/python/ModuleProject-main/dataset/seq_V_17363197.npy')
piece_data = np.load('C:/Users/user/Desktop/python/ModuleProject-main/dataset/seq_piece_1736319783.npy')

print("Good shape:", good_data.shape)
print("Five shape:", five_data.shape)
print("Fit shape:", fit_data.shape)
print("V shape:", V_data.shape)
print("Piece shape:", piece_data.shape)

# Concatenate data
data = np.concatenate([good_data, five_data, fit_data, V_data, piece_data], axis=0)
print("Concatenated data shape:", data.shape)

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/user/Desktop/python/ModuleProject-main/dataset/seq_seq_fit_1736319783.npy'

In [ ]:
x_data = data[:, :, :-1]
labels = data[:, 0, -1]

print(x_data.shape)
print(labels.shape)

In [ ]:
from tensorflow.keras.utils import to_categorical

y_data = to_categorical(labels, num_classes=len(actions))
y_data.shape

In [ ]:
from sklearn.model_selection import train_test_split

x_data = x_data.astype(np.float32)
y_data = y_data.astype(np.float32)

x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.1, random_state=2021)

print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout, LeakyReLU

model = Sequential([
    GRU(64, input_shape=x_train.shape[1:3], return_sequences=False),  # GRU 레이어
    LeakyReLU(alpha=0.3),  # LeakyReLU 적용
    Dropout(0.5),  # 50% 드롭아웃
    Dense(32),
    LeakyReLU(alpha=0.3),  # LeakyReLU 적용
    Dropout(0.3),  # 30% 드롭아웃
    Dense(len(actions), activation='softmax')  # 출력층
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

history = model.fit(
    x_train,
    y_train,
    validation_data=(x_val, y_val),
    epochs=200,
    callbacks=[
        ModelCheckpoint('models/model.h5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto'),
        ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=50, verbose=1, mode='auto')
    ]
)

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix
from tensorflow.keras.models import load_model

model = load_model('models/model.h5')

y_pred = model.predict(x_val)

multilabel_confusion_matrix(np.argmax(y_val, axis=1), np.argmax(y_pred, axis=1))